In [ ]:
!pip install ctransformers==0.2.27 transformers==4.40.2 langchain==0.1.20 langchain-community==0.0.38 torch pypdf sentence-transformers faiss-cpu openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9

In [ ]:
pip install gpt4all==2.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 14.8 MB/s eta 0:00:00


In [ ]:
!pip install streamlit==1.36.0 pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.9 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts.prompt import PromptTemplate
import time
# OpenAI API key setup
openai.api_key = 'copy your API key here'

# Path to the vector store
vector_db_path = "/content/drive/MyDrive/vectorstores/db_faiss"

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def get_intent(sentence):
  messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Identify the intent of the following sentence in one or two words or three words: '{sentence}'."}
  ]
  response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    max_tokens=50,
    temperature=0.5
)

  intent = response.choices[0].message['content'].strip().lower()
  return intent

def get_sentence_embedding(sentence, model):
  return model.encode([sentence])[0]

def find_most_relevant_intent(sentence, intent_list):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  identified_intent = get_intent(sentence)
  print(f"Identified Intent: {identified_intent}")

  identified_intent_embedding = get_sentence_embedding(identified_intent, model)

  best_intent = None
  best_score = 0

  for intent in intent_list:
    intent_embedding = get_sentence_embedding(intent, model)
    score = cosine_similarity([identified_intent_embedding], [intent_embedding])[0][0]
    print(f"Intent: {intent}, Similarity Score: {score:.4f}")
    if score > best_score:
        best_intent = intent
        best_score = score

  return best_intent, best_score
def extract_entities(sentence):
  messages = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": f"Extract the name, age, and gender (if mentioned) from the following sentence: '{sentence}'."}
]
  response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    max_tokens=100,
    temperature=0.5
)
  entities = response.choices[0].message['content'].strip()
  return entities
Zung_questions = [
    {"question": "Tôi cảm thấy nóng nảy và lo âu hơn thường lệ", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi cảm thấy sợ vô cớ", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi dễ bối rối và cảm thấy hoảng sợ", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi cảm thấy như bị ngã và vỡ ra từng mảnh", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi cảm thấy mọi thứ đều tốt và không có điều gì xấu sẽ xảy ra", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tay và chân tôi lắc lư, run lên", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi đang khó chịu vì đau đầu, đau cổ, đau lưng", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi cảm thấy yếu và dễ mệt mỏi", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi cảm thấy bình tĩnh và có thể ngồi yên một cách dễ dàng", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi cảm thấy tim mình đập nhanh", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi đang khó chịu vì cơn hoa mắt chóng mặt", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi bị ngất và có lúc cảm thấy gần như thế", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi có thể thở ra, hít vào một cách dễ dàng", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi cảm thấy tê buốt, như có kiến bò ở đầu ngón tay, ngón chân", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi đang khó chịu vì đau dạ dày và đầy bụng", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi luôn cần phải đi đái", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Bàn tay tôi thường khô và ấm", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Mặt tôi thường nóng và đỏ", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi ngủ dễ dàng và luôn có một giấc ngủ tốt", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}},
    {"question": "Tôi thường có ác mộng", "options": {"Không có": 0, "Đôi khi": 1, "Phần lớn thời gian": 2, "Hầu hết hoặc tất cả thời gian": 3}}
]

Beck_questions = [
    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không cảm thấy buồn.": 0, "Nhiều lúc tôi cảm thấy chán hoặc buồn.": 1, "Lúc nào tôi cũng cảm thấy chán hoặc buồn và tôi không thể dừng lại được.": 2, "Lúc nào tôi cũng cảm thấy buồn và bất hạnh đến mức hoàn toàn đau khổ.": 2, "Tôi rất buồn hoặc rất bất hạnh và khổ sở đến mức không thể chịu được.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi hoàn toàn không bi quan và nản lòng về tương lai.": 0, "Tôi cảm thấy nản lòng về tương lai hơn trước đây.": 1, "Tôi cảm thấy mình chẳng có gì mong đợi ở tương lai cả.": 2, "Tôi cảm thấy sẽ không bao giờ khắc phục được những điều phiền muộn của tôi.": 2, "Tôi cảm thấy tương lai tuyệt vọng và tình hình chỉ có thể tiếp tục xấu đi hoặc không thể cải thiện được.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không cảm thấy như bị thất bại.": 0, "Tôi thấy mình thất bại nhiều hơn những người khác.": 1, "Tôi cảm thấy đã hoàn thành rất ít điều đáng giá hoặc đã hoàn thành rất ít điều có ý nghĩa.": 2, "Nhìn lại cuộc đời, tôi thấy mình đã có quá nhiều thất bại.": 2, "Tôi cảm thấy mình là một người hoàn toàn thất bại.": 3, "Tôi tự cảm thấy hoàn toàn thất bại trong vai trò của tôi (bố, mẹ, chồng, vợ…).": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi hoàn toàn không bất mãn.": 0, "Tôi còn thích thú với những điều mà trước đây tôi vẫn thường ưa thích.": 0, "Tôi luôn luôn cảm thấy buồn.": 1, "Tôi ít thấy thích những điều mà tôi vẫn thường ưa thích trước đây.": 1, "Tôi không thỏa mãn về bất kỳ cái gì nữa.": 2, "Tôi rất ít thích thú về những điều trước đây tôi vẫn thường ưa thích.": 2, "Tôi không còn chút thích thú nào nữa.": 3, "Tôi không hài lòng với mọi cái.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi hoàn toàn không cảm thấy có tội lỗi gì ghê gớm cả.": 0, "Phần nhiều những việc tôi đã làm tôi đều cảm thấy có tội.": 1, "Phần lớn thời gian tôi cảm thấy mình tồi hoặc không xứng đáng.": 1, "Tôi cảm thấy mình hoàn toàn có tội.": 2, "Giờ đây tôi luôn cảm thấy trên thực tế mình tồi hoặc không xứng đáng.": 2, "Lúc nào tôi cũng cảm thấy mình có tội.": 3, "Tôi cảm thấy như là tôi rất tồi hoặc vô dụng.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không cảm thấy đang bị trừng phạt.": 0, "Tôi cảm thấy có thể mình sẽ bị trừng phạt.": 1, "Tôi cảm thấy một cái gì xấu có thể đến với tôi.": 1, "Tôi mong chờ bị trừng phạt.": 2, "Tôi cảm thấy mình sẽ bị trừng phạt.": 2, "Tôi cảm thấy mình đang bị trừng phạt.": 3, "Tôi muốn bị trừng phạt.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi thấy bản thân mình vẫn như trước kia hoặc tôi không cảm thấy thất vọng với bản thân.": 0, "Tôi thất vọng với bản thân, tôi không còn tin tưởng vào bản thân hoặc tôi không thích bản thân.": 1, "Tôi thất vọng với bản thân hoặc tôi ghê tởm bản thân.": 2, "Tôi ghét bản thân mình hoặc tôi căm thù bản thân.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không phê phán hoặc đổ lỗi cho bản thân hơn trước kia.": 0, "Tôi không tự cảm thấy một chút nào xấu hơn bất kể ai.": 0, "Tôi phê phán bản thân mình nhiều hơn trước kia.": 1, "Tôi tự chê mình về sự yếu đuối và lỗi lầm của bản thân.": 1, "Tôi phê phán bản thân về tất cả những lỗi lầm của mình.": 2, "Tôi khiển trách mình vì những lỗi lầm của bản thân.": 2, "Tôi đổ lỗi cho bản thân về tất cả mọi điều tồi tệ xảy ra.": 3, "Tôi khiển trách mình về mọi điều xấu xảy đến.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không có ý nghĩ tự sát.": 0, "Tôi không có bất kỳ ý nghĩ gì làm tổn hại bản thân.": 0, "Tôi có ý nghĩ tự sát nhưng không thực hiện.": 1, "Tôi có những ý nghĩ làm tổn hại bản thân nhưng tôi thường không thực hiện chúng.": 1, "Tôi muốn tự sát.": 2, "Tôi cảm thấy giá mà tôi chết thì tốt hơn.": 2, "Tôi cảm thấy gia đình tôi sẽ tốt hơn nếu tôi chết.": 2, "Tôi có dự định rõ ràng để tự sát.": 2, "Nếu có cơ hội tôi sẽ tự sát.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không khóc nhiều hơn trước kia.": 0, "Hiện nay tôi hay khóc nhiều hơn trước.": 1, "Tôi thường khóc vì những điều nhỏ nhặt.": 2, "Hiện tại tôi luôn luôn khóc, tôi không thể dừng được.": 2, "Trước đây thỉnh thoảng tôi vẫn khóc, nhưng hiện tại tôi không thể khóc được chút nào mặc dù tôi muốn khóc.": 3, "Tôi thấy muốn khóc nhưng không thể khóc được.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không dễ bồn chồn và căng thẳng hơn thường lệ.": 0, "Hiện nay tôi không dễ bị kích thích hơn trước.": 0, "Tôi cảm thấy dễ bồn chồn và căng thẳng hơn thường lệ.": 1, "Tôi bực mình hoặc phát cáu dễ dàng hơn trước.": 1, "Tôi cảm thấy bồn chồn và căng thẳng đến mức khó có thể ngồi yên được.": 2, "Tôi luôn luôn cảm thấy dễ phát cáu.": 2, "Tôi thấy rất bồn chồn và kích động đến mức phải đi lại liên tục hoặc làm việc gì đó.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không mất sự quan tâm đến những người xung quanh hoặc các hoạt động khác.": 0, "Tôi ít quan tâm đến mọi người, mọi việc xung quanh hơn trước.": 1, "Tôi mất hầu hết sự quan tâm đến mọi người, mọi việc xung quanh và ít có cảm tình với họ.": 2, "Tôi không còn quan tâm đến bất kỳ điều gì nữa.": 3, "Tôi hoàn toàn không còn quan tâm đến người khác và không cần đến họ chút nào.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi quyết định mọi việc cũng tốt như trước.": 0, "Tôi thấy khó quyết định mọi việc hơn trước.": 1, "Tôi thấy khó quyết định mọi việc hơn trước rất nhiều.": 2, "Không có sự giúp đỡ, tôi không thể quyết định gì được nữa.": 2, "Tôi chẳng còn có thể quyết định được việc gì nữa.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không cảm thấy mình là người vô dụng.": 0, "Tôi không tự thấy mình tệ hơn trước.": 0, "Tôi cảm thấy mình tệ hơn, vô dụng hơn so với trước đây.": 1, "Tôi cảm thấy mình có thể trở thành một gánh nặng cho người khác.": 1, "Tôi cảm thấy mình là một gánh nặng cho người khác.": 2, "Tôi cảm thấy mình vô dụng.": 2, "Tôi cảm thấy mình hoàn toàn vô dụng.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không mệt mỏi hơn thường lệ.": 0, "Tôi cảm thấy mệt mỏi dễ dàng hơn trước.": 1, "Tôi cảm thấy mệt mỏi nhiều hơn trước.": 2, "Tôi mệt mỏi suốt cả ngày và không có khả năng thực hiện bất kỳ điều gì cả.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi vẫn thấy thích ăn và không có thay đổi gì về thói quen ăn uống.": 0, "Tôi ít thấy thích ăn hơn trước.": 1, "Tôi có thay đổi trong thói quen ăn uống và tôi không thích ăn nữa.": 1, "Tôi ăn ít hơn trước.": 1, "Tôi thấy bữa ăn không ngon miệng nữa.": 2, "Thực sự tôi thấy chán ăn.": 2, "Tôi có thay đổi rõ rệt về thói quen ăn uống.": 2, "Gần đây tôi ăn nhiều hơn trước.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Gần đây tôi không giảm cân hoặc thay đổi cân nặng.": 0, "Tôi cảm thấy cân nặng giảm xuống hoặc tăng lên.": 1, "Tôi giảm hoặc tăng cân đáng kể trong thời gian gần đây mà không có lý do gì rõ ràng.": 1, "Gần đây tôi giảm cân rất nhiều (trên 5kg).": 2, "Gần đây tôi tăng cân rất nhiều (trên 5kg).": 2}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không bị bất kỳ thay đổi nào về giấc ngủ.": 0, "Tôi thấy khó ngủ hơn so với trước kia.": 1, "Tôi thấy ngủ nhiều hơn trước đây.": 1, "Tôi thường xuyên thức giấc vào ban đêm và khó ngủ lại.": 1, "Tôi cảm thấy mệt mỏi khi thức dậy.": 2, "Tôi thấy cần phải ngủ nhiều hơn.": 2, "Tôi thường xuyên thức dậy sớm hơn 2 tiếng so với thường lệ.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi quan tâm đến sức khỏe của mình như trước.": 0, "Tôi có sự lo lắng về sức khỏe hơn trước.": 1, "Tôi quá lo lắng về sức khỏe của mình.": 2, "Tôi luôn luôn lo lắng về sức khỏe của mình và tập trung vào những triệu chứng đau đớn của cơ thể.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Tôi không mệt mỏi hơn trước.": 0, "Tôi không lo lắng về sức khỏe hơn trước.": 0, "Tôi dễ mệt mỏi hơn trước.": 1, "Tôi có lo lắng về những đau đớn hoặc những khó chịu ở dạ dày hoặc táo bón và những cảm giác của cơ thể.": 1, "Hầu như làm bất kỳ việc gì tôi cũng thấy mệt mỏi.": 2, "Tôi quá lo lắng về sức khỏe của tôi, tôi cảm thấy thế nào và điều gì đó đến nổi tôi rất khó suy nghĩ gì thêm nữa.": 2, "Tôi quá mệt mỏi khi làm bất kỳ việc gì.": 3, "Tôi hoàn toàn bị thu hút vào những cảm giác của tôi.": 3}},

    {"question": "Hãy chọn 1 ý đúng nhất với bản thân", "options": {"Ham muốn tình dục của tôi không giảm.": 0, "Ham muốn tình dục của tôi giảm nhẹ.": 1, "Ham muốn tình dục của tôi giảm nhiều.": 2, "Tôi hoàn toàn không có ham muốn tình dục.": 3}}
]

# Load LLM using ChatOpenAI
def load_llm():
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai.api_key)
    return llm

# Create prompt template
def create_prompt(template):
    prompt = PromptTemplate(template=template, input_variables=["history", "context","question"])
    return prompt

# Create simple chain with memory
def create_qa_chain(prompt, llm, db, memory):
    llm_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        verbose=True,
        retriever=db.as_retriever(search_kwargs={"k": 26}, max_tokens_limit=1024),
        return_source_documents=False,
        chain_type_kwargs={
          "verbose": True,
          "prompt": prompt,
          "memory": ConversationBufferMemory(
            memory_key="history",
            input_key="question"),
        },
        memory=memory
    )
    return llm_chain

# Read from VectorDB
def read_vectors_db():
    # Embedding model
    embedding_model = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
    return db
def in_data (x) :
    st.session_state.messages.append(
              {
                  "role": "assistant",
                  "content": x
              }
            )

# Start experiment
db = read_vectors_db()

# Load LLM from OpenAI GPT
llm = load_llm()

# Create conversation summary memory
summary_memory = ConversationBufferMemory(llm=llm)
template = """
Sử dụng thông tin dưới đây để trả lời câu hỏi, kết hợp với lịch sử đoạn chat.
------
{context}
------
{history}
------
{question}
"""
prompt = create_prompt(template)
intent_list = ["aswering_question_mental_health_psychology","test"]
intent_list_no = ["depression_check", "anxiety_check"]
# Lưu lịch sử trò chuyện
st.title('Mental Health Care chatbot ')


if "messages" not in  st.session_state:
    st.session_state.messages = []
if "depresion_check" not in st.session_state :
    st.session_state.depresion_check = False
if 'llm_chain' not in st.session_state:
    llm_chain = create_qa_chain(prompt, llm, db, summary_memory)
    st.session_state.llm_chain = llm_chain
if 'answers_BECK' not in st.session_state:
        st.session_state.answers_BECK = [None] * len(Beck_questions)
if 'answers_ZUNG' not in st.session_state:
        st.session_state.answers_ZUNG = [None] * len(Zung_questions)
if 'total_score_BECK' not in st.session_state:
        st.session_state.total_score_BECK = 0
if 'total_score_ZUNG' not in st.session_state:
        st.session_state.total_score_ZUNG = 0
if 'anxiety_check' not in st.session_state:
    st.session_state.anxiety_check = False
if 'start' not in st.session_state :
    st.session_state.start = False
################ GT #####################################
if st.session_state.start == False :
    st.session_state.messages.append(
                {
                    "role": "assistant",
                    "content": '''Chào mừng bạn đến với hệ thống hỗ trợ tâm lý! Đây là nơi bạn có thể dễ dàng kiểm tra và theo dõi tình trạng tâm lý của mình. Chúng tôi cung cấp các bài kiểm tra chuyên sâu như BECK để đánh giá mức độ trầm cảm và ZUNG để kiểm tra mức độ lo âu. Bạn cũng có thể trả lời các câu hỏi tâm lý để tự đánh giá tình trạng của mình. Hãy bắt đầu ngay để hiểu rõ hơn về sức khỏe tinh thần của bạn!'''
                }
              )
    st.session_state.start = True
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])


 ##################################### BECK CHECK #################################################
if st.session_state.depresion_check == True :
      with st.form(key='quiz_form'):
        st.write("Vui lòng trả lời tất cả các câu hỏi và nhấn 'Nộp' để xem kết quả.")

        for i, question in enumerate(Beck_questions):
            st.write(f"Đề mục {i + 1}: {question['question']}")
            st.session_state.answers_BECK[i] = st.radio(
                label=f"Câu hỏi {i + 1}",
                options=list(question['options'].keys()),
                key=f"q_{i}"
            )

        # Nút nộp
        submit_button = st.form_submit_button(label='Nộp')
        if submit_button:
            # Tính tổng điểm
            total_score = 0
            for i, answer in enumerate(st.session_state.answers_BECK):
                if answer:
                    total_score += Beck_questions[i]['options'].get(answer, 0)

            st.session_state.total_score_BECK = total_score
            in_data(f"Tổng điểm của bạn là: {st.session_state.total_score_BECK}")

            # Đánh giá mức độ trầm cảm dựa trên tổng điểm
            if total_score < 14:
                in_data("Đánh giá: Không biểu hiện trầm cảm.")
            elif 14 <= total_score <= 19:
                in_data("Đánh giá: Trầm cảm nhẹ.")
            elif 20 <= total_score <= 29:
                in_data("Đánh giá: Trầm cảm vừa.")
            else:
                in_data("Đánh giá: Trầm cảm nặng.")
            st.session_state.depresion_check = False


#####################################3 ANXIETY #############################################################

if st.session_state.anxiety_check == True :
      with st.form(key='quiz_form'):
        st.write("Vui lòng trả lời tất cả các câu hỏi và nhấn 'Nộp' để xem kết quả.")

        for i, question in enumerate(Zung_questions):
            st.write(f"Đề mục {i + 1}: {question['question']}")
            st.session_state.answers_ZUNG[i] = st.radio(
                label=f"Câu hỏi {i + 1}",
                options=list(question['options'].keys()),
                key=f"q_{i}"
            )

        # Nút nộp
        submit_button = st.form_submit_button(label='Nộp')
        if submit_button:
            # Tính tổng điểm
            total_score = 0
            for i, answer in enumerate(st.session_state.answers_ZUNG):
                if answer:
                    total_score += Zung_questions[i]['options'].get(answer, 0)

            st.session_state.total_score_ZUNG = total_score
            in_data(f"Tổng điểm của bạn là: {st.session_state.total_score_ZUNG}")

            # Đánh giá mức độ trầm cảm dựa trên tổng điểm
            if total_score <= 40:
                in_data("Đánh giá: Không lo âu")
            elif 41 <= total_score <= 50:
                in_data("Đánh giá: Lo âu mức độ nhe")
            elif 51 <= total_score <= 60:
                in_data("Đánh giá: Lo âu mức độ vừa")
            elif 61 <= total_score <= 70:
                in_data("Đánh giá: Lo âu mức độ nặng")
            else:
                in_data("Đánh giá: Lo âu mức độ rất nặng")
            st.session_state.anxiety_check = False


if st.session_state.depresion_check == 0 and st.session_state.anxiety_check == 0 :
  if prompt := st.chat_input("Hãy nhập vào yêu cầu?"):
            st.session_state.messages.append(
                {
                    "role": "user",
                    "content": prompt
                }
            )
            with st.chat_message('user'):
                st.markdown(prompt)

            ## Code chính ask_question
            best_intent, best_score = find_most_relevant_intent (prompt, intent_list)
            if (best_intent == "aswering_question_mental_health_psychology" or "?" in prompt) and "kiểm tra" not in prompt :
              response = st.session_state.llm_chain.invoke({"query": prompt})
              with st.chat_message('assistant'):
                full_res = ""
                holder = st.empty()
                for word in response['result'].split():
                    full_res += word + " "
                    time.sleep(0.05)
                    holder.markdown(full_res + "▌")
                holder.markdown(response['result'])
              st.session_state.messages.append(
                {
                    "role": "assistant",
                    "content": full_res
                }
              )
            else :
                best_intent1, best_score = find_most_relevant_intent (prompt, intent_list_no)
                if best_intent1 == "depression_check":
                  in_data ('Thang Đo Trầm Cảm Beck (Beck Depression Inventory - BDI) là một công cụ giúp bạn tự đánh giá mức độ trầm cảm của mình. Bài test gồm 21 câu hỏi, mỗi câu hỏi sẽ hỏi về các cảm giác hoặc triệu chứng mà bạn có thể trải qua, như cảm thấy buồn bã, mệt mỏi, hay mất hứng thú trong cuộc sống. Bạn chỉ cần chọn mức độ phù hợp với bản thân từ các lựa chọn có sẵn. Kết quả sẽ giúp bạn hiểu rõ hơn về trạng thái tâm lý hiện tại và có thể là cơ sở để tìm kiếm sự tư vấn hoặc hỗ trợ nếu cần thiết.')
                  st.session_state.depresion_check = True
                  st.rerun()
                else :
                  in_data ('Thang Đo Lo Âu Zung (Zung Self-Rating Anxiety Scale - SAS) là một công cụ giúp bạn tự đánh giá mức độ lo âu của mình. Bài test gồm 20 câu hỏi, mỗi câu hỏi sẽ hỏi về các cảm giác hoặc triệu chứng lo âu mà bạn có thể trải qua, như cảm thấy căng thẳng, bồn chồn, khó ngủ, hay tim đập nhanh. Bạn chỉ cần chọn mức độ phù hợp với bản thân từ các lựa chọn có sẵn. Kết quả sẽ giúp bạn hiểu rõ hơn về mức độ lo âu hiện tại và có thể là cơ sở để tìm kiếm sự tư vấn hoặc hỗ trợ nếu cần thiết.')
                  st.session_state.anxiety_check = True
                  st.rerun ()


Writing app.py


In [ ]:
import threading
from pyngrok import ngrok

# Đặt mã thông báo của bạn từ Ngrok dashboard
ngrok.set_auth_token('set your token here')

def run_streamlit():
    !streamlit run app.py --server.port 8514

def start_ngrok():
    public_url = ngrok.connect(8514)
    print(f"Public URL: {public_url}")

threading.Thread(target=run_streamlit).start()
start_ngrok()


In [ ]:
pip show streamlit

Name: streamlit
Version: 1.37.1
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, rich, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 


In [ ]:
pip install streamlit

In [ ]:
!pip install paramiko scp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 32.4 MB/s eta 0:00:00


In [ ]:
!ssh-keygen -R 103.153.64.170

Cannot stat /root/.ssh/known_hosts: No such file or directory


In [ ]:
import paramiko
from scp import SCPClient

# Thông tin kết nối SSH
hostname = 'mentalchatbot.site'
port = 22
username = 'myuser'  # Your username
password = 'mypassword'  # Your password
remote_file_path = 'app.py'  

# Kết nối SSH
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, port, username, password)

# Tạo file app.py trên local để demo
with open('app.py', 'w', encoding='utf-8') as file:
    file.write('print("Xin chào, chào mừng đến với Mental Chatbot!")\n')

# Tải file Python lên server
scp = SCPClient(ssh.get_transport())
scp.put('app.py', remote_file_path)
scp.close()

print("Đã tải file app.py lên server.")

# Chạy Streamlit trên server
stdin, stdout, stderr = ssh.exec_command(f'streamlit run {remote_file_path} --server.port 8510')
print(stdout.read().decode())
print(stderr.read().decode())

# Đóng kết nối SSH
ssh.close()

print("Đã chạy ứng dụng Streamlit trên server.")


gaierror: [Errno -2] Name or service not known

In [ ]:
import openai
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
openai.api_key = 'copy your API here'

def get_intent(sentence):
  messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Identify the intent of the following sentence in one or two words or three words: '{sentence}'."}
  ]
  response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    max_tokens=50,
    temperature=0.5
)

  intent = response.choices[0].message['content'].strip().lower()
  return intent

def get_sentence_embedding(sentence, model):
  return model.encode([sentence])[0]

def find_most_relevant_intent(sentence, intent_list):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  identified_intent = get_intent(sentence)
  print(f"Identified Intent: {identified_intent}")

  identified_intent_embedding = get_sentence_embedding(identified_intent, model)

  best_intent = None
  best_score = 0

  for intent in intent_list:
    intent_embedding = get_sentence_embedding(intent, model)
    score = cosine_similarity([identified_intent_embedding], [intent_embedding])[0][0]
    print(f"Intent: {intent}, Similarity Score: {score:.4f}")
    if score > best_score:
        best_intent = intent
        best_score = score

  return best_intent, best_score
def extract_entities(sentence):
  messages = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": f"Extract the name, age, and gender (if mentioned) from the following sentence: '{sentence}'."}
]
  response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    max_tokens=100,
    temperature=0.5
)

  entities = response.choices[0].message['content'].strip()
  return entities

intent_list = ["ask_question", "check_scale", "other"]
sentence = ""
best_intent, best_score = find_most_relevant_intent(sentence, intent_list)
print(f"The most relevant intent is: {best_intent} with a similarity score of: {best_score:.4f}")

entities = extract_entities(sentence)
print(f"Extracted Entities: {entities}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(




2024-07-18 09:17:42.353 Port 8510 is already in use
Identified Intent: your message appears to be empty. could you please provide the sentence you want me to identify the intent of?
Intent: ask_question, Similarity Score: 0.2064
Intent: check_scale, Similarity Score: 0.0591
Intent: other, Similarity Score: 0.1237
The most relevant intent is: ask_question with a similarity score of: 0.2064
Extracted Entities: The sentence you provided is empty. Please provide a sentence with the desired information.


In [ ]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts.prompt import PromptTemplate

# OpenAI API key setup
openai.api_key = 'your API key here'

# Path to the vector store
vector_db_path = "/content/drive/MyDrive/vectorstores/db_faiss"


# Load LLM using ChatOpenAI
def load_llm():
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai.api_key)
    return llm

# Create prompt template
def create_prompt(template):
    prompt = PromptTemplate(template=template, input_variables=["history", "context","question"])
    return prompt

# Create simple chain with memory
def create_qa_chain(prompt, llm, db, memory):
    llm_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        verbose=True,
        retriever=db.as_retriever(search_kwargs={"k": 12}, max_tokens_limit=2048),
        return_source_documents=False,
        chain_type_kwargs={
          "verbose": True,
          "prompt": prompt,
          "memory": ConversationBufferMemory(
            memory_key="history",
            input_key="question"),
        },
        memory=memory
    )
    return llm_chain

# Read from VectorDB
def read_vectors_db():
    # Embedding model
    embedding_model = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
    return db

# Start experiment
db = read_vectors_db()

# Load LLM from OpenAI GPT
llm = load_llm()

# Create conversation summary memory
summary_memory = ConversationBufferMemory(llm=llm)
template = """
Sử dụng thông tin dưới đây để trả lời câu hỏi, kết hợp với lịch sử đoạn chat. Nếu không biết hãy nói không biết đừng cố tạo ra câu trả lời.
------
{context}
------
{history}
------
{question}
"""
prompt = create_prompt(template)
llm_chain = create_qa_chain(prompt, llm, db, summary_memory)
# Run the chain
question = "Những thuộc tính cơ bản của khí chất là gì?"
response = llm_chain.invoke({"query": question})
print(response['result'])  # In ra chỉ giá trị của khóa 'result'




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Sử dụng thông tin dưới đây để trả lời câu hỏi, kết hợp với lịch sử đoạn chat. Nếu không biết hãy nói không biết đừng cố tạo ra câu trả lời.
------
chí điều đó có thể khác biệt với những gì bạn bè nghĩ và cậu có thể bị trêu chọc.
Như bạn thấy, trí thông minh nội tâm cũng giống như sự tự nhận thức. Và đây là một
trong những loại kiến thức quan trọng nhất bạn cần có. Thật khó mà tin được rằng

Khoái cảm bao giờ cũng khiến bạn lệ thuộc vào người 
khác. Hạnh phúc giúp bạn có ụdslashược một chút tự chủ nhưng không hoàn toàn. Chỉ có niềm an lạc mới là tự do, nó không chia cắt bất cứ ụdslashiều gì, không chia cắt ai; ụdslashó là ụdslashời sống duy nhất của bạn, là bản chất sâu xa nhất của bạn. 
TỪ BỀ MẶT ĐẾN TRUNG TÂM
Đức Thích Ca Mâu Ni nói:

Trí Thông Minh Tương Tác Cá Nhân Là Gì?
Bạn yêu mến mọi người và thể hiện điều đó bằng nhiều cách khác nhau. 

In [ ]:
# Hàm để tính tổng điểm Beck Depression Inventory (BDI)
def calculate_bdi_score(answers):

    total_score = sum(answers)
    return total_score

# Hàm để thực hiện bài kiểm tra trầm cảm
def conduct_bdi_test():
    # Tạo mảng chứa câu hỏi và các lựa chọn
    questions = [
        "1.\n   1. Tôi không cảm thấy buồn \n   2. Nhiều lúc tôi cảm thấy chán hoặc buồn\n   3. Lúc nào tôi cũng cảm thấy chán hoặc buồn và tôi không thể dừng lại được \n   4. Lúc nào tôi cũng cảm thấy buồn và bất hạnh đến mức hoàn toàn đau khổ \n   5. Tôi rất buồn hoặc rất bất hạnh và khổ sở đến mức không thể chịu được \n",
        "2.\n   1. Tôi hoàn toàn không bi quan và nản lòng về tương lai \n   2. Tôi cảm thấy nản lòng về tương lai hơn trước đây \n   3. Tôi cảm thấy mình chẳng có gì mong đợi ở tương lai cả \n   4. Tôi cảm thấy sẽ không bao giờ khắc phục được những điều phiền muộn của tôi \n   5. Tôi cảm thấy tương lai tuyệt vọng và tình hình chỉ có thể tiếp tục xấu đi hoặc không thể cải thiện được \n",
        "3.\n   1. Tôi không cảm thấy như bị thất bại\n   2. Tôi thấy mình thất bại nhiều hơn những người khác\n   3.Tôi cảm thấy đã hoàn thành rất ít điều đáng giá hoặc đã hoàn thành rất ít điều có ý nghĩa\n   4. Nhìn lại cuộc đời, tôi thấy mình đã có quá nhiều thất bại\n   5. Tôi cảm thấy mình là một người hoàn toàn thất bại\n   6. Tôi tự cảm thấy hoàn toàn thất bại trong vai trò của tôi (bố, mẹ, chồng, vợ…)\n",
        "4.\n   1. Tôi hoàn toàn không bất mãn \n   2. Tôi còn thích thú với những điều mà trước đây tôi vẫn thường ưa thích\n   3. Tôi luôn luôn cảm thấy buồn\n   4. Tôi ít thấy thích những điều mà tôi vẫn thường ưa thích trước đây\n   5. Tôi không thỏa mãn về bất kỳ cái gì nữa\n   6. Tôi rất ít thích thú về những điều trước đây tôi vẫn thường ưa thích\n   7. Tôi không còn chút thích thú nào nữa\n   8. Tôi không hài lòng với mọi cái\n",
        "5.\n   1. Tôi hoàn toàn không cảm thấy có tội lỗi gì ghê gớm cả\n   2. Phần nhiều những việc tôi đã làm tôi đều cảm thấy có tội\n   3. Phần lớn thời gian tôi cảm thấy mình tồi hoặc không xứng đáng\n   4. Tôi cảm thấy mình hoàn toàn có tội\n   5. Giờ đây tôi luôn cảm thấy trên thực tế mình tồi hoặc không xứng đáng\n   6. Lúc nào tôi cũng cảm thấy mình có tội\n   7. Tôi cảm thấy như là tôi rất tồi hoặc vô dụng\n",
        "6.\n   1. Tôi không cảm thấy đang bị trừng phạt\n   2. Tôi cảm thấy có thể mình sẽ bị trừng phạt\n   3. Tôi cảm thấy một cái gì xấu có thể đến với tôi\n   4. Tôi mong chờ bị trừng phạt\n   5. Tôi cảm thấy mình sẽ bị trừng phạt\n   6. Tôi cảm thấy mình đang bị trừng phạt\n   7. Tôi muốn bị trừng phạt\n",
        "7.\n   1. Tôi thấy bản thân mình vẫn như trước kia hoặc tôi không cảm thấy thất vọng với bản thân\n   2. Tôi thất vọng với bản thân, tôi không còn tin tưởng vào bản thân hoặc tôi không thích bản thân \n   3. Tôi thất vọng với bản thân hoặc Tôi ghê tởm bản thân \n   4. Tôi ghét bản thân mình hoặc Tôi căm thù bản thân\n",
        "8.\n   1. Tôi không phê phán hoặc đổ lỗi cho bản thân hơn trước kia\n   2. Tôi không tự cảm thấy một chút nào xấu hơn bất kể ai\n   3.Tôi phê phán bản thân mình nhiều hơn trước kia\n   4. Tôi tự chê mình về sự yếu đuối và lỗi lầm của bản thân\n   5. Tôi phê phán bản thân về tất cả những lỗi lầm của mình\n   6. Tôi khiển trách mình vì những lỗi lầm của bản thân\n   7. Tôi đổ lỗi cho bản thân về tất cả mọi điều tồi tệ xảy ra\n   8. Tôi khiển trách mình về mọi điều xấu xảy đến\n",
        "9.\n   1. Tôi không có ý nghĩ tự sát\n   2. Tôi không có bất kỳ ý nghĩ gì làm tổn hại bản thân\n   3. Tôi có ý nghĩ tự sát nhưng không thực hiện\n   4. Tôi có những ý nghĩ làm tổn hại bản thân nhưng tôi thường không thực hiện chúng\n   5. Tôi muốn tự sát \n   6. Tôi cảm thấy giá mà tôi chết thì tốt hơn \n   7. Tôi cảm thấy gia đình tôi sẽ tốt hơn nếu tôi chết\n   8. Tôi có dự định rõ ràng để tự sát\n   9. Nếu có cơ hội tôi sẽ tự sát\n",
        "10.\n    1. Tôi không khóc nhiều hơn trước kia\n    2. Hiện nay tôi hay khóc nhiều hơn trước\n    3. Tôi thường khóc vì những điều nhỏ nhặt\n    4. Hiện tại tôi luôn luôn khóc, tôi không thể dừng được\n    5. Tôi thấy muốn khóc nhưng không thể khóc được\n    6. Trước đây thỉnh thoảng tôi vẫn khóc, nhưng hiện tại tôi không thể khóc được chút nào mặc dù tôi muốn khóc\n",
        "11.\n    1. Tôi không dễ bồn chồn và căng thẳng hơn thường lệ\n    2. Hiện nay tôi không dễ bị kích thích hơn trước\n     3. Tôi cảm thấy dễ bồn chồn và căng thẳng hơn thường lệ\n    4. Tôi bực mình hoặc phát cáu dễ dàng hơn trước\n    5. Tôi cảm thấy bồn chồn và căng thẳng đến mức khó có thể ngồi yên được\n    6. Tôi luôn luôn cảm thấy dễ phát cáu\n    7. Tôi thấy rất bồn chồn và kích động đến mức phải đi lại liên tục hoặc làm việc gì đó\n",
        "12.\n    1. Tôi không mất sự quan tâm đến những người xung quanh hoặc các hoạt động khác\n    2. Tôi ít quan tâm đến những người xung quanh hơn trước\n    3. Tôi ít quan tâm đến những hoạt động khác hơn trước\n    4. Tôi cảm thấy ít thân thiện hơn với mọi người xung quanh\n    5. Tôi ít quan tâm đến các hoạt động trước đây tôi vẫn thường ưa thích\n    6. Tôi không quan tâm đến các hoạt động trước đây tôi vẫn thường làm nữa\n    7. Tôi không quan tâm đến các hoạt động trước đây tôi vẫn thường yêu thích nữa\n",
        "13.\n    1. Tôi có thể quyết định dễ dàng như trước\n    2. Tôi thấy khó khăn hơn trước trong việc đưa ra quyết định\n    3. Tôi thấy rất khó khăn khi quyết định điều gì đó\n    4. Tôi cảm thấy không thể đưa ra quyết định gì cả\n    5. Tôi không thể quyết định điều gì cả\n    6. Tôi không thể quyết định điều gì được cả\n",
        "14.\n    1. Tôi không cảm thấy kém hấp dẫn hơn trước\n    2. Tôi cảm thấy kém hấp dẫn hơn trước\n    3. Tôi không cảm thấy mình có sức hấp dẫn gì nữa\n    4. Tôi cảm thấy không hấp dẫn chút nào\n",
        "15.\n    1. Tôi vẫn làm việc và các sinh hoạt hằng ngày của mình như trước\n    2. Tôi thấy khó khăn khi bắt đầu làm một việc gì đó\n    3. Tôi khó khăn khi làm một việc gì đó\n    4. Tôi cần cố gắng nhiều hơn bình thường để có thể bắt đầu làm việc\n    5. Tôi thấy mình cần cố gắng rất nhiều khi làm một việc gì đó\n    6. Tôi không làm được việc gì nữa cả\n    7. Tôi không thể làm việc gì cả\n",
        "16.\n    1. Tôi vẫn ngủ ngon giấc như trước\n    2. Tôi khó đi vào giấc ngủ hơn trước\n    3. Tôi khó ngủ và không thể ngủ ngon giấc\n    4. Tôi khó ngủ hơn và thức giấc nhiều lần trong đêm\n    5. Tôi thức dậy nhiều lần trong đêm và thấy khó đi vào giấc ngủ lại\n    6. Tôi thức giấc quá sớm buổi sáng và khó ngủ lại được\n    7. Tôi không ngủ được chút nào cả\n    8. Tôi thường thức dậy sớm hơn bình thường 1 hoặc 2 giờ và không thể ngủ lại được\n    9. Tôi thức giấc từ 2 hoặc 3 giờ sáng và không thể ngủ lại được nữa\n",
        "17.\n    1. Tôi không cảm thấy mệt mỏi hơn thường lệ\n    2. Tôi cảm thấy mệt mỏi hơn bình thường\n    3. Tôi thấy mình không thể nghỉ ngơi được\n    4. Tôi thấy rất mệt mỏi hoặc rã rời\n    5. Tôi mệt mỏi đến mức không làm được việc gì cả\n    6. Tôi mệt mỏi hoặc cạn kiệt năng lượng đến mức không thể làm được điều gì\n",
        "18.\n    1. Khẩu vị của tôi không tồi hơn trước\n    2. Tôi thấy ăn uống không ngon miệng như trước\n    3. Tôi thấy khẩu vị của mình tồi hơn bình thường\n    4. Khẩu vị của tôi rất kém, tôi phải ép mình ăn\n    5. Tôi không cảm thấy thèm ăn chút nào\n",
        "19.\n    1. Tôi vẫn tăng cân hoặc giảm cân bình thường\n    2. Tôi giảm hơn 2 kg mà không có chủ định\n    3. Tôi tăng hơn 2 kg mà không có chủ định\n    4. Tôi giảm hơn 4 kg mà không có chủ định\n    5. Tôi tăng hơn 4 kg mà không có chủ định\n",
        "20.\n    1. Tôi không quan tâm về sức khỏe của mình hơn trước\n    2. Tôi thấy lo lắng về sức khỏe của mình hơn trước\n    3. Tôi thấy rất lo lắng về sức khỏe của mình\n    4. Tôi cảm thấy mình không khỏe mạnh và lo lắng nhiều về sức khỏe của mình\n    5. Tôi thấy rất lo lắng về sức khỏe của mình đến mức không thể làm được điều gì khác\n",
        "21.\n    1. Tôi không thấy những thay đổi về sở thích tình dục của mình\n    2. Tôi ít hứng thú hơn với sở thích tình dục của mình\n    3. Tôi thấy hứng thú với sở thích tình dục của mình kém hơn trước\n    4. Tôi hoàn toàn không có sở thích tình dục nữa\n    5. Tôi mất hoàn toàn sở thích tình dục\n",
    ]

    # Thu thập câu trả lời
    answers = []
    for i, question in enumerate(questions):
        print(f"{question}")
        while True:
            try:
                answer = int(input(f"Chọn đáp án cho câu hỏi {i + 1}: "))
                if 1 <= answer <= 4:
                    answers.append(answer)
                    break
                else:
                    print("Vui lòng nhập giá trị từ 1 đến 4.")
            except ValueError:
                print("Vui lòng nhập một số hợp lệ.")

    # Tính điểm BDI
    print (answers)
    score = calculate_bdi_score(answers)
    return score

In [ ]:
class MentalHealthChatbot:
    def __init__(self):
        self.greeting_responses = ["Xin chào! Tôi là chatbot chăm sóc sức khỏe tinh thần. Tôi có thể giúp bạn với các chức năng sau:\n1. Kiểm tra trạng thái tâm lý\n2. Trả lời câu hỏi về sức khỏe tinh thần\n3. Cung cấp các gợi ý công việc"]
        self.goodbye_responses = ["Tạm biệt! Hãy nhớ chăm sóc sức khỏe tinh thần của bạn.", "Hẹn gặp lại! Chúc bạn một ngày tốt lành.", "Tạm biệt! Hãy giữ sức khỏe."]
        self.menu = [
            "1. Kiểm tra trạng thái tâm lý",
            "2. Trả lời câu hỏi về sức khỏe tinh thần",
            "3. Gợi ý công việc"
        ]

    def get_response(self, user_input):
        user_input = user_input.lower()
        list_intent = ['ask_question', 'check_scale', 'other']
        best_intent, best_score = find_most_relevant_intent( user_input, intent_list)
        print (best_intent)
        if best_intent == "ask_question" :
          response = llm_chain.invoke( user_input)
          print (response)

    def get_greeting(self):
        return self.greeting_responses[0]

    def get_goodbye(self):
        return self.goodbye_responses[0]

    def get_menu(self):
        return "\n".join(self.menu)

    def check_mental_state(self):
        # Đây là nơi bạn có thể thêm logic kiểm tra trạng thái tâm lý
        return "Hãy mô tả tâm trạng của bạn hôm nay và tôi sẽ giúp bạn kiểm tra trạng thái tâm lý của mình."

    def mental_health_faq(self):
        # Đây là nơi bạn có thể thêm các câu trả lời cho các câu hỏi thường gặp về sức khỏe tinh thần
        return "Bạn có thể hỏi tôi về bất kỳ vấn đề nào liên quan đến sức khỏe tinh thần. Ví dụ: làm thế nào để giảm stress?"

    def get_suggestions(self):
        return "\n".join(self.menu)

    def get_default_response(self):
        return "aaaa"


# Khởi tạo chatbot
chatbot = MentalHealthChatbot()

# Giới thiệu bản thân khi bắt đầu
print("Chatbot:", chatbot.get_greeting())

# Vòng lặp để chatbot liên tục nhận và trả lời câu hỏi
while True:
    user_input = input("Bạn: ")
    if user_input.lower() in ["tạm biệt", "bye"]:
        print("Chatbot:", chatbot.get_response(user_input))
        break
    else:
        print("Chatbot:", chatbot.get_response(user_input))


Chatbot: Xin chào! Tôi là chatbot chăm sóc sức khỏe tinh thần. Tôi có thể giúp bạn với các chức năng sau:
1. Kiểm tra trạng thái tâm lý
2. Trả lời câu hỏi về sức khỏe tinh thần
3. Cung cấp các gợi ý công việc


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Identified Intent: check weight
Intent: ask_question, Similarity Score: 0.1806
Intent: check_scale, Similarity Score: 0.5529
Intent: other, Similarity Score: 0.0556
check_scale
Chatbot: aaaa


In [ ]:
pip install gemini


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 65.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 11.7 MB/s

In [ ]:
pip install python-gemini-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.3 MB/s eta 0:00:00


In [ ]:
pip install -U google-generativeai

In [ ]:
import google.generativeai as genai
key = "API key here"
genai.configure (
api_key = key
)
model = genai.GenerativeModel ('gemini-pro')
chat = model.start_chat (history = [])
print (chat.send_message ("trầm cảm là gì ?").text)

Trầm cảm là một tình trạng sức khỏe tâm thần đặc trưng bởi cảm xúc buồn chán, mất hứng thú và rối loạn khác ảnh hưởng tiêu cực đến cuộc sống hàng ngày.

**Triệu chứng:**

* Cảm thấy buồn chán hoặc tuyệt vọng không lý do
* Mất hứng thú với các hoạt động bình thường
* Giảm năng lượng và cảm giác mệt mỏi
* Khó tập trung hoặc đưa ra quyết định
* Thay đổi về chế độ ăn uống hoặc giấc ngủ
* Cảm giác tội lỗi hoặc vô giá trị
* Suy nghĩ tự tử hoặc tự làm hại bản thân
* Các cơn đau nhức hoặc các triệu chứng thể chất khác mà không thể giải thích được

**Nguyên nhân:**

Trầm cảm có thể do nhiều yếu tố gây ra, bao gồm:

* Yếu tố di truyền
* Sự mất mát hoặc chấn thương
* Các sự kiện căng thẳng trong cuộc sống
* Bất thường về hóa học não
* Tình trạng sức khỏe thể chất

**Các loại trầm cảm:**

* **Trầm cảm lớn:** Tình trạng trầm cảm nghiêm trọng nhất, gây ra các triệu chứng nghiêm trọng can thiệp vào cuộc sống hàng ngày.
* **Rối loạn trầm cảm dai dẳng (PDD):** Một tình trạng trầm cảm kéo dài ít nhất 2 

In [ ]:
pip install -U langchain-google-genai==1.0.7

In [ ]:
import getpass
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("API key here")

In [ ]:
pip install langchain==0.2.6 langchain_core==0.2.10 langchain_google_genai==1.0.7


  Using cached langchain-0.2.6-py3-none-any.whl (975 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 4.6 MB/s eta 0:00:00
  Using cached langsmith-0.1.83-py3-none-any.whl (127 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.92
    Uninstalling langsmith-0.0.92:
      Successfully uninstalled langsmith-0.0.92
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.0.7
    Uninstalling langchain-core-0.0.7:
      Successfully uninstalled langchain-core-0.0.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.2
    Uninstalling langchain-0.2.2:
      Successfully uninstalled langchain-0.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.10 which is in

In [ ]:
!export API_KEY=<AIP Key here>

In [ ]:
import google.generativeai as genai
import os

# Replace 'YOUR_ACTUAL_API_KEY' with your real API key
api_key = 'API key here'

genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-1.5-flash')


In [ ]:
pip install langchain_core==0.2.3 langchain

INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 13.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.2
    Uninstalling langchain-text-splitters-0.2.2:
      Successfully uninstalled langchain-text-splitters-0.2.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.6
    Uninstalling langchain-0.2.6:
      Successfully uninstalled langchain-0.2.6


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (/usr/local/lib/python3.10/dist-packages/langchain_core/language_models/chat_models.py)

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
pip install langchain 0.0.1

  Preparing metadata (setup.py) ... done
  Using cached langchain_core-0.2.11-py3-none-any.whl (337 kB)
  Using cached langsmith-0.1.83-py3-none-any.whl (127 kB)
  Created wheel for 0.0.1: filename=0.0.1-0.0.1-py2.py3-none-any.whl size=20096 sha256=6871042e6498fb2ce0755f79f4e045106d2d77096297a941148587086f8c9d93
  Stored in directory: /root/.cache/pip/wheels/87/f2/ef/f38643635c7596b51eee6b5bb8992d2b102fcf64de4bd805a6
Successfully built 0.0.1
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.92
    Uninstalling langsmith-0.0.92:
      Successfully uninstalled langsmith-0.0.92
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.1
    Uninstalling langchain-core-0.1.1:
      Successfully uninstalled langchain-core-0.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.38 r

In [ ]:
pip install langchain_core==0.1.1

  Using cached langchain_core-0.1.1-py3-none-any.whl (190 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
ERROR: Operation cancelled by user


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (/usr/local/lib/python3.10/dist-packages/langchain_core/language_models/chat_models.py)